In [1]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from datetime import datetime, timedelta

In [2]:
datastocks = pd.read_csv('data_snp_stocks.csv')
snp500 = pd.read_csv('index.csv')
datastocks['date'] = pd.to_datetime(datastocks['date'], format = '%Y-%m-%d')
datastocks.set_index('date', inplace=True)
tickers = datastocks['Name'].unique()

In [3]:
datastocks.dtypes

open      float64
high      float64
low       float64
close     float64
volume      int64
Name       object
dtype: object

In [4]:
datastocks.head()

,open,high,low,close,volume,Name
date,,,,,,
2013-02-08,15.07,15.12,14.63,14.75,8407500,AAL
2013-02-11,14.89,15.01,14.26,14.46,8882000,AAL
2013-02-12,14.45,14.51,14.10,14.27,8126000,AAL
2013-02-13,14.30,14.94,14.25,14.66,10259500,AAL
2013-02-14,14.94,14.96,13.16,13.99,31879900,AAL


In [ ]:
datastocks.isna().sum()

In [ ]:
#data.sort_values(by=['Name', 'date'], inplace=True)
datastocks['LogReturn'] = datastocks.groupby('Name')['close'].transform(lambda x: np.log(x / x.shift(1)))

In [ ]:
data['LogReturn'].isna().sum()

In [ ]:
data = data[data['LogReturn'].notna()]

In [ ]:
start_date = '2017-01-01'
# Convert start_date to a datetime object
start_date_obj = datetime.strptime(start_date, '%Y-%m-%d')

# Add 180 days to start_date_obj
end_date_obj = start_date_obj + timedelta(days=261)

# Convert end_date_obj back to a string
end_date = end_date_obj.strftime('%Y-%m-%d')
#end_date = 

In [ ]:
data180 = pd.DataFrame()
for ticker in tickers:
    data180 = pd.concat([data180, data[(data['Name'] == ticker) & (data.index >= start_date) & (data.index <= end_date)]])
data180

To perform the rolling regression we would need to solve the following equation:

$r_i -r_f = \alpha_i + \beta_i (R_M - r_f) + e_i$

Where:

$r_i$ is the return of the stock $i$ \
$r_f$ is the return of the risk free asset \
$\alpha_i$ is the intercept of the regression, or the excess return \
$\beta_i$ is the regression coefficient \
$R_M$ is the return of the market (log-return) \
$e_i$ is the specific risk of stock $i$


In [ ]:
data180.groupby('Name').count()

In [ ]:
df_coeff = pd.DataFrame(columns = ['Ticker', 'alpha', 'beta', 'r_squared', 'specific_risk'])

